In [ ]:
#make  a simple script to send mail to selector

In [1]:
import os

In [3]:
import win32com.client as win32

In [8]:
olApp =win32.Dispatch('Outlook.Aplication')
olns = olApp.GetNameSpace('MAPI')


In [ ]:
mailItem = olApp.CreateItem(0)
mailItem.Subject ='Dummy Email'
mailItem.BodyFormat =1
mailItem.Body ="hello world"
mailItem.To -'<Recipient mail>'

In [ ]:
mailItem.Attachments.Add(os.path.join(os.getcwd(),'nameoffile'))

In [ ]:
mailItem.Display()

In [ ]:
mailItem.Save()
mailItem.Send()